# Fitting a FLMP
In this notebook, we will fit a Fuzzy Logical Model of Perception (FLMP) to audiovisual speech perception data.

In [1]:
import numpy as np
from os import path
from glob import glob

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

from scipy.special import comb
from scipy.stats import binom
from scipy.optimize import minimize, LinearConstraint, Bounds

## Prepare Data
The data consists of five text files, each containing seven rows and five columns, where:
- Row 1: Audiotorial data
- Row 2: Visual data
- Rows 3-7: Audiovisual data
    - a combination of rows 1 and 2
    - visual goes from 'b' (row 3) to 'd' (row 7) 
    - audio goes from 'b' (col 1) to 'd' (col 5)

In [2]:
# get paths to data files
file_paths = glob("./data/*.txt")

# load all data into a single array
data = np.array([np.loadtxt(fname) for fname in file_paths])
N, M, K = data.shape

# define number of samples for each subject
n_samples = 24 

# split into the three different data types
# data_A = data[:, 0, :]
# data_V = data[:, 1, :]
# data_AV = data[:, 2:, :]

## Fit FLMP

In [3]:
# create softmax function
def baseline_softmax(x):
    x = np.concatenate([
        np.array([x]).flatten(), [0]
    ])
    e = np.exp(x)
    return e / e.sum()

# create our own pmf function
def binomial_pmf(k, n, p):
    return comb(n, k) * np.power(p, k) * np.power(1 - p, n - k)

In [7]:
def objective_function(theta, data, n_samples=24):
    # extract audio and visual parameters
    # theta_A = theta[0:(K-1)]
    # theta_V = theta[(K-1): ]
    theta_A = theta[0:K]
    theta_V = theta[K: ]

    # get probabilities for audio and visual
    # p_A = baseline_softmax(theta_A).reshape(-1,1)
    # p_V = baseline_softmax(theta_V).reshape(-1,1)
    p_A = np.array([baseline_softmax(t)[0] for t in theta_A]).reshape(-1,1)
    p_V = np.array([baseline_softmax(t)[0] for t in theta_V]).reshape(-1,1)
    
    # compute audiovisual probabilities by
    # taking the outer product for all combinations of audio and visual
    p_AV = (p_A @ p_V.T) / (p_A @ p_V.T + (1 - p_A) @ (1 - p_V).T)
    
    # compute the log-likelihoods
    probs = np.vstack([p_A.T, p_V.T, p_AV])
    L = np.log(binomial_pmf(data, n_samples, probs)).sum()
    # L_A = binom.logpmf(data[0], n_samples, p_A).flatten()
    # L_V = binom.logpmf(data[1], n_samples, p_V).flatten()
    # L_AV = binom.logpmf(data[2:], n_samples, p_AV).flatten()
    # L = np.concatenate([L_A, L_V, L_AV]).sum()

    # return the negative log-likelihood
    return -L

In [8]:
def flmp_fit(data, n_samples):
    """Perform FLMP fit to data"""
    # K_ = K - 1
    K_ = K
    theta = np.zeros(K_*2)
    opt_result = minimize(objective_function, theta, args=(data, n_samples))
    objective, theta_A, theta_V = (
        opt_result.fun, 
        (opt_result.x[0:K_]), 
        (opt_result.x[K_:])
    )
    return objective, theta_A, theta_V

In [9]:
# perform FLMP fit for each subject
theta_A = np.zeros((K, N))
theta_V = np.zeros((K, N))
neg_L = np.zeros(K)
for i in range(K):
    obj, tA, tV = flmp_fit(data[i], n_samples)

    # save results
    neg_L[i] = obj
    theta_A[i, :] = tA 
    theta_V[i, :] = tV
    # theta_A[i, :-1] = tA
    # theta_V[i, :-1] = tV

    # print results
    print(f'\nResults for subject {i+1}')
    print(f'Negative log-likelihood: {neg_L[i]}')
    print("Audio:", theta_A[i])
    print("Visual:", theta_V[i])


Results for subject 1
Negative log-likelihood: 70.25765786050826
Audio: [-4.78771542 -1.95605364 -0.25823502  1.43526796  1.67202829]
Visual: [-1.11834396  0.51165128  1.54670804  4.28161049  5.03519686]

Results for subject 2
Negative log-likelihood: 41.7976564702329
Audio: [-6.55473261 -1.91458401  0.34489206  3.09946715  3.87804737]
Visual: [-1.6323042   1.07303217  2.55000152  4.14860438  6.12230226]

Results for subject 3
Negative log-likelihood: 68.33041239058295
Audio: [-2.95926905 -1.13433544 -0.10648988  1.29981559  1.60598464]
Visual: [-1.53452301 -0.3698892   0.86865851  3.57910917  5.65092501]

Results for subject 4
Negative log-likelihood: 72.3148065626844
Audio: [-2.66352712 -1.96654471 -1.27082507  0.82708934  1.69431479]
Visual: [-3.69983135 -2.16532762 -1.17804284  2.62584348  4.25103027]

Results for subject 5
Negative log-likelihood: 73.02387779051162
Audio: [-4.43200284 -2.15618122 -0.97970257  0.64690831  1.04513991]
Visual: [-3.56948525 -1.18540161  1.06838789  4